In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim

# 데이터 읽기
data = pd.read_csv("diabetes.csv")

# 독립 변수와 종속 변수 설정
X = data.drop("Outcome", axis=1)
y = data["Outcome"]

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 데이터 정규화
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 텐서 변환
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.FloatTensor(y_train.values)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.FloatTensor(y_test.values)

# 회귀 모델 정의
class RegressionModel(nn.Module):
    def __init__(self, input_dim):
        super(RegressionModel, self).__init__()
        self.fc = nn.Linear(input_dim, 1)  # 연속값 예측
    
    def forward(self, x):
        return self.fc(x)

# 모델 초기화
input_dim = X_train_tensor.shape[1]
model = RegressionModel(input_dim)

# 손실 함수와 옵티마이저 설정
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# 모델 학습
epochs = 100
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs.view(-1), y_train_tensor)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

# 평가
model.eval()
with torch.no_grad():
    predictions = model(X_test_tensor).view(-1)
    test_loss = criterion(predictions, y_test_tensor)
    print(f"Test Loss: {test_loss.item():.4f}")


Epoch [10/100], Loss: 0.6619
Epoch [20/100], Loss: 0.4167
Epoch [30/100], Loss: 0.3016
Epoch [40/100], Loss: 0.2396
Epoch [50/100], Loss: 0.2013
Epoch [60/100], Loss: 0.1787
Epoch [70/100], Loss: 0.1667
Epoch [80/100], Loss: 0.1611
Epoch [90/100], Loss: 0.1587
Epoch [100/100], Loss: 0.1578
Test Loss: 0.1669
